In [1]:
from allennlp_models.pretrained import load_predictor
from allennlp_models.pretrained import get_pretrained_models

import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect

from checklist.pred_wrapper import PredictorWrapper
import pandas as pd

from nltk.corpus import wordnet as wn
import random

In [2]:
nsamples = 100

In [3]:
def switch_topic(sentence):
    string_split = sentence.split()
    topic = string_split[-2:]
    topic[0] = topic[0].capitalize()
    topic = words = [w.replace(".", ",") for w in topic]
    not_topic = string_split[:-2]
    if not_topic[0] != "I":
        not_topic[0] = not_topic[0].lower()
    rebuilt_sentence_list = topic + not_topic
    rebuilt_sentence = " ".join(rebuilt_sentence_list)+"."
    return rebuilt_sentence

def get_pp_tags(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        pp_tags = output['verbs'][1]['tags']
    else:
        pp_tags = []
    return pp_tags

def get_pp_descr(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        pp_descr = output['verbs'][1]['description']
    else:
        pp_descr = []
    return pp_descr

def find_final_min_one_arg(list_of_arg):
    if list_of_arg:
        element = list_of_arg[-3]
    else:
        element = 'missed_verb'
    return element

def find_first_arg(list_of_arg):
    if list_of_arg:
        element = list_of_arg[0]
    else:
        element = 'missed_verb'
    return element

def get_loc_tags(sentence):
    output = srl_predictor.predict(sentence)
    loc_tags = output['verbs'][1]['tags'][-2]
    return loc_tags

In [4]:
food = wn.synset('food.n.02')
food = list(set([w for s in food.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
singular_food =  [x.lower() for x in food if "_" not in x ]

# random.seed(18)
# sampled_list = random.sample(singular_food, 100)
# print(sampled_list)

/Users/syb/opt/anaconda3/envs/the/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:579: UserWarning: Discarded redundant search for Synset('lunch_meat.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/syb/opt/anaconda3/envs/the/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:579: UserWarning: Discarded redundant search for Synset('headcheese.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/syb/opt/anaconda3/envs/the/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:579: UserWarning: Discarded redundant search for Synset('haslet.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/syb/opt/anaconda3/envs/the/lib/python3.9/site-packages/nltk/corpus/reader/wordnet.py:579: UserWarning: Discarded redundant search for Synset('leg_of_lamb.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/Users/syb/opt/anaconda3/envs/the/lib/python3.9/site-packages/n

In [5]:
list_topicalization_neg = []
editor = Editor()

test_data = editor.template("I wont eat that {singular_food}.",
                            singular_food = singular_food,
                            meta=True, 
                            seed=42,  
                            nsamples=nsamples,
                            remove_duplicates=True)
list_topicalization_neg.append(test_data['data'])

#flatten list
list_topicalization_neg = [x for xs in list_topicalization_neg for x in xs]

list_topicalization = []

editor = Editor()
test_data = editor.template("I will eat that {singular_food}.",
                            singular_food = singular_food,
                            meta=True, 
                            seed=42,  
                            nsamples=nsamples,
                            remove_duplicates=True)
list_topicalization.append(test_data['data'])

#flatten list
list_topicalization = [x for xs in list_topicalization for x in xs]

In [6]:
srl_predictor = load_predictor('structured-prediction-srl')

df = pd.DataFrame(list_topicalization_neg)
df[1] = df[0].apply(switch_topic)
df.rename(columns = {0:'original', 1:'switched'}, inplace = True)

df['tags_original'] = df['original'].apply(get_pp_tags)
df['descr_original'] = df['original'].apply(get_pp_descr)

df['tags_switched'] = df['switched'].apply(get_pp_tags)
df['descr_switched'] = df['switched'].apply(get_pp_descr)

df['final-1_arg_original'] = df['tags_original'].apply(find_final_min_one_arg)
df['first_arg_switch'] = df['tags_switched'].apply(find_first_arg)

df['first_arg_switch'] = df['first_arg_switch'].str.replace('B-', '')
df['first_arg_switch'] = df['first_arg_switch'].str.replace('I-', '')

df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('I-', '')
df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('B-', '')

df['orginal_switched_match'] = df['final-1_arg_original'] == df['first_arg_switch']

df['original'].to_csv("../datasets/topicalization/negated.txt", header=None, index=False)
df.to_csv("../results/topicalization/results_negated_bilstm.csv")

df_error_neg_bilstm = df[df['orginal_switched_match'] == False][['descr_original', 'descr_switched']]

lerc is not a registered model.


In [7]:
df = pd.DataFrame(list_topicalization)
df[1] = df[0].apply(switch_topic)
df.rename(columns = {0:'original', 1:'switched'}, inplace = True)

df['tags_original'] = df['original'].apply(get_pp_tags)
df['descr_original'] = df['original'].apply(get_pp_descr)

df['tags_switched'] = df['switched'].apply(get_pp_tags)
df['descr_switched'] = df['switched'].apply(get_pp_descr)

df['final-1_arg_original'] = df['tags_original'].apply(find_final_min_one_arg)
df['first_arg_switch'] = df['tags_switched'].apply(find_first_arg)

df['first_arg_switch'] = df['first_arg_switch'].str.replace('B-', '')
df['first_arg_switch'] = df['first_arg_switch'].str.replace('I-', '')

df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('I-', '')
df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('B-', '')

df['orginal_switched_match'] = df['final-1_arg_original'] == df['first_arg_switch']
df.to_csv("../results/topicalization/results_simple_bilstm.csv")

df_error_simple_bilstm = df[df['orginal_switched_match'] == False][['descr_original', 'descr_switched']]

In [8]:
srl_predictor = load_predictor('structured-prediction-srl-bert')

df = pd.DataFrame(list_topicalization)
df[1] = df[0].apply(switch_topic)
df.rename(columns = {0:'original', 1:'switched'}, inplace = True)

df['tags_original'] = df['original'].apply(get_pp_tags)
df['descr_original'] = df['original'].apply(get_pp_descr)

df['tags_switched'] = df['switched'].apply(get_pp_tags)
df['descr_switched'] = df['switched'].apply(get_pp_descr)

df['final-1_arg_original'] = df['tags_original'].apply(find_final_min_one_arg)
df['first_arg_switch'] = df['tags_switched'].apply(find_first_arg)

df['first_arg_switch'] = df['first_arg_switch'].str.replace('B-', '')
df['first_arg_switch'] = df['first_arg_switch'].str.replace('I-', '')

df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('I-', '')
df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('B-', '')

df['orginal_switched_match'] = df['final-1_arg_original'] == df['first_arg_switch']
df.to_csv("../results/topicalization/results_simple_bert.csv")

df_error_simple_bert = df[df['orginal_switched_match'] == False][['descr_original', 'descr_switched']]

lerc is not a registered model.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
df = pd.DataFrame(list_topicalization_neg)
df[1] = df[0].apply(switch_topic)
df.rename(columns = {0:'original', 1:'switched'}, inplace = True)

df['tags_original'] = df['original'].apply(get_pp_tags)
df['descr_original'] = df['original'].apply(get_pp_descr)

df['tags_switched'] = df['switched'].apply(get_pp_tags)
df['descr_switched'] = df['switched'].apply(get_pp_descr)

df['final-1_arg_original'] = df['tags_original'].apply(find_final_min_one_arg)
df['first_arg_switch'] = df['tags_switched'].apply(find_first_arg)

df['first_arg_switch'] = df['first_arg_switch'].str.replace('B-', '')
df['first_arg_switch'] = df['first_arg_switch'].str.replace('I-', '')

df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('I-', '')
df['final-1_arg_original'] = df['final-1_arg_original'].str.replace('B-', '')

df['orginal_switched_match'] = df['final-1_arg_original'] == df['first_arg_switch']

df_error_neg_bert = df[df['orginal_switched_match'] == False][['descr_original', 'descr_switched']]
df.to_csv("../results/topicalization/results_neg_bert.csv")

In [12]:
failure_simple_bert = len(df_error_simple_bert)/len(df)*100
failure_neg_bert = len(df_error_neg_bert)/len(df)*100
failure_simple_bilstm = len(df_error_simple_bilstm)/len(df)*100
failure_neg_bilstm = len(df_error_neg_bilstm)/len(df)*100

In [13]:
print("BERT")
print("Simple: ", failure_simple_bert)
print("Negated: ", failure_neg_bert)
print("bilstm")
print("Simple: ", failure_simple_bilstm)
print("Negated: ", failure_neg_bilstm)

BERT
Simple:  10.0
Negated:  21.0
bilstm
Simple:  4.0
Negated:  3.0
